Examples of how to use the Isophotes.py file from the Auriga_Analysis package to find isophotes in surface brightness maps and fit to ellipses to find isophote centres

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import Auriga_Analysis as A

plt.rcParams['text.usetex'] = True

In [ ]:
x = 3  #index of magnitude. 3 = k-band
f = 10**6  #make pc
shrinking_circles_centres = []

snap = 1516
snapobj = A.SnapshotData.load_snapshot(snap, 4, ["Coordinates","GFM_StellarPhotometrics", "Velocities", "Masses"])
coords,magnitudes,velocities, masses = snapobj.get_results()

radius = A.GroupData.group_data(snap, 0, "Group/Group_R_Crit200").return_detail()
centre = A.GroupData.group_data(snap, 0, "Subhalo/SubhaloPos").return_detail()

new_coords, new_vels, coords, vels, masses, magnitude = A.util.align(centre, coords, velocities, masses, radius, 0.5, idx = None, other = [magnitudes], to_print = False)
coord2d = new_coords[:,1:3]   #just look at y-z plane

#apply shrinking circles algorithm, weighting by magnitude?
pos, mass, no_iterations, rad, ss_centre = A.ShrinkingSpheres.shrinking_circles(coord2d, masses,0.35*radius, 0.75,5000, [0,0])

shrinking_circles_centres.append(ss_centre)

In [ ]:
def stellar_hist(snap, bin_no, histogram_width, sigma, axes):

    b = histogram_width
    pix_size = ((histogram_width)*(10**6)/bin_no)**2
    time = A.util.time(snap)

    results = A.SnapshotData.load_snapshot(snap, 4, ["Coordinates","GFM_StellarPhotometrics", "Velocities", "Masses"])
    coordinates, magnitudes, velocities, masses = results.get_results()
    radius = A.GroupData.group_data(snap, 0, "Group/Group_R_Crit200").return_detail()
    centre = A.GroupData.group_data(snap, 0, "Subhalo/SubhaloPos").return_detail()
    
    new_coords, new_vels, coords, vels, masses, magnitude = A.util.align(centre, coordinates, velocities, masses, radius, 0.1, idx = None, other = [magnitudes], to_print = False)

    n, xedges, yedges = np.histogram2d(new_coords[:,axes[0]], new_coords[:,axes[1]], bins=(500,500), range = [[-b,b], [-b, b]])
    xbin = 0.5 * (xedges[:-1] + xedges[1:])
    ybin = 0.5 * (yedges[:-1] + yedges[1:])
    xc, yc = np.meshgrid(xbin, ybin)
    fig, ax1 = plt.subplots(figsize=(5, 5))
    ax1.set_aspect(1)
    ax1.contourf( xc*1000, yc*1000, np.log10(n.T), cmap='magma')

    plt.title('Time = {} Gyr'.format(np.round(time,2)))
    plt.xlabel('x / kpc')
    plt.ylabel('y / kpc')


In [ ]:
matrices = np.load('Data/Variable_rotation_matrices.npy')
rotmat_snaps = list(np.load('Data/Rotmat_snapshots.npy'))

In [ ]:
def snapshot_data(snap):

    time = A.util.time(snap)

    results = A.SnapshotData.load_snapshot(snap, 4, ["Coordinates","GFM_StellarPhotometrics", "Velocities", "Masses"])
    coordinates, magnitudes, velocities, masses = results.get_results()
    radius = A.GroupData.group_data(snap, 0, "Group/Group_R_Crit200").return_detail()
    centre = A.GroupData.group_data(snap, 0, "Subhalo/SubhaloPos").return_detail()

    new_coords, new_vels, coords, vels, masses, magnitude = A.util.align(centre, coordinates, velocities, masses, radius, 0.5, idx = None, other = [magnitudes], to_print = False)

    return new_coords, new_vels, magnitude[0], masses, time

In [ ]:
def snapshot_data_aligned(snap):
    time = A.util.time(snap)

    results = A.SnapshotData.load_snapshot(snap, 4, ["Coordinates","GFM_StellarPhotometrics", "Velocities", "Masses"])
    coordinates, magnitudes, velocities, masses = results.get_results()
    radius = A.GroupData.group_data(snap, 0, "Group/Group_R_Crit200").return_detail()
    centre = A.GroupData.group_data(snap, 0, "Subhalo/SubhaloPos").return_detail()

    if snap in rotmat_snaps:
        centred = A.util.CentreOnHalo(coordinates, centre)
        velocities, bulk_velocity = A.util.remove_bulk_velocity(centred, velocities, masses, idx = None, radialcut = 0.1*radius)
        rotmat = matrices[rotmat_snaps.index(snap)]
        new_coords = np.dot(centred, rotmat)
        new_vels = np.dot(velocities, rotmat)
    
    else:
        new_coords, new_vels, coords, vels, masses, magnitudes = A.util.align(centre, coordinates, velocities, masses, radius, 0.1, idx = None, other = [magnitudes], to_print = False)
        magnitudes = magnitudes[0]
        
    return new_coords, new_vels, magnitudes, masses, time

In [ ]:
snap = 2999

coords, vels, magnitudes, masses, time = snapshot_data(snap)
n,xc, yc = A.Isophotes.make_SB_map(coords, 0.035, 128, magnitudes, 3, 1.5, 10**3, time, show_plot = True)

In [ ]:
f = 10**3
b =25
iso_magnitudes = [15, 16, 17, 18, 19, 20, 21]
centre_mags = [[],[],[],[],[],[]]

snap = 2000

coords, vels, magnitudes, masses, time = snapshot_data_aligned(snap)
n,xc, yc = A.Isophotes.make_SB_map(coords, 0.035, 128, magnitudes, 3, 1.5, 10**3, time, show_plot = True)
   
for i in range(len(iso_magnitudes)):
    magnitude = iso_magnitudes[i]
    coords = A.Isophotes.isophotes(n, xc, yc, 0.01*magnitude, magnitude)
    plt.scatter(coords[:,0]*f, coords[:,1]*f, label = 'magnitude ={}'.format(magnitude), s = 1, color = 'black')
    x,y,centre = A.Isophotes.tilted_ellipse_LS(coords, 1000)
    centre_mags[i].append(centre)

In [ ]:
f = 10**3
b =20

isophote_magnitudes = [17, 18, 19, 20, 21]
centre_mags = [[] for _ in isophote_magnitudes]
distances = [[] for _ in isophote_magnitudes]
times = []

for j in range(1200,1400,2):
    snap = j
    if snap%100 ==0:
        print(snap)
    coords, vels, magnitudes, masses, time = snapshot_data_aligned(snap)
    times.append(time)
    n,xc, yc = A.Isophotes.make_SB_map(coords, 0.024, 128, magnitudes, 3, 1.5, 10**3, time, show_plot = False)
    
    for i in range(len(isophote_magnitudes)):
        magnitude = isophote_magnitudes[i]
        coords = A.Isophotes.isophotes(n, xc, yc, 0.01*magnitude, magnitude)

        if len(coords) > 0:
            x,y,centre = A.Isophotes.tilted_ellipse_LS(coords, 1000)
            distances[i].append(np.sqrt(centre[0]**2+centre[1]**2)/0.6777)

        centre_mags[i].append(centre)
        plt.scatter(coords[:,0]*f, coords[:,1]*f, label = 'magnitude ={}'.format(magnitude), s = 1, color = 'black')
        plt.scatter(x*f,y*f, color = 'red', s = 0.5)

plt.title('Time = {} Gyr'.format(np.round(time, 2)),fontsize = 14)
plt.xlabel('x / kpc')
plt.ylabel('y / kpc')
plt.xlim(-b,b)
plt.ylim(-b,b)

plt.close()
centres = [np.array(centre) for centre in centre_mags]